In [4]:
# Import necessary packages
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import torchshow as ts
import IPython.display as ipd
import os
import pandas as pd
import torchaudio
import torchshow as ts
import librosa

In [5]:
class AvianNatureSounds(Dataset):
    def __init__(self, annotation_file=None, root_dir='./',mel_spectrogram = None,mode='wav'):
        self.annotation_file = pd.read_csv(annotation_file).sort_values('fileName')
        self.root_dir = root_dir
        self.mel_transformation = mel_spectrogram
        self.AmplitudeToDB = torchaudio.transforms.AmplitudeToDB()
        self.mode = mode


    def __len__(self):
        return len(self.annotation_file)

    def __getitem__(self, index):
        if self.mode == 'wav':
            audio_sample_path = os.path.join(self.root_dir,os.listdir(self.root_dir)[index])
            label = self.annotation_file.iloc[index]['habitat']
            signal, sr = torchaudio.load(audio_sample_path)

            return (signal, sr), label , audio_sample_path
        
        elif self.mode == 'mel':
            audio_sample_path = os.path.join(self.root_dir,os.listdir(self.root_dir)[index])
            label = self.annotation_file.iloc[index]['habitat']
            signal, sr = torchaudio.load(audio_sample_path)
            signal = self.AmplitudeToDB(self.mel_transformation(signal))

            # signal[:, 65:] = 0

            # return (signal, sr), label
            return signal, label
        
        elif self.mode == 'stft': 
            audio_sample_path = os.path.join(self.root_dir,os.listdir(self.root_dir)[index])
            label = self.annotation_file.iloc[index]['habitat']
            signal, sr = torchaudio.load(audio_sample_path)

            stft = torch.stft(signal, n_fft=1024, hop_length=512, normalized=True, return_complex=True)

            mag = self.AmplitudeToDB(torch.abs(stft))
            phase = torch.angle(stft)

            return torch.cat([mag,phase],dim=0), label
        
        elif self.mode == 'stft_mag': 
            audio_sample_path = os.path.join(self.root_dir,os.listdir(self.root_dir)[index])
            label = self.annotation_file.iloc[index]['habitat']
            signal, sr = torchaudio.load(audio_sample_path)

            stft = torch.stft(signal, n_fft=1024, hop_length=4096*2, normalized=True, return_complex=True)

            mag = self.AmplitudeToDB(torch.abs(stft))

            return mag, label
        
        elif self.mode == 'stft_phase': 
            audio_sample_path = os.path.join(self.root_dir,os.listdir(self.root_dir)[index])
            label = self.annotation_file.iloc[index]['habitat']
            signal, sr = torchaudio.load(audio_sample_path)

            stft = torch.stft(signal, n_fft=1024, hop_length=4096*2, normalized=True, return_complex=True) # see https://dsp.stackexchange.com/a/258

            phase = torch.angle(stft)

            return phase, label


        else:
            raise NotImplementedError

In [6]:
class LinearInterpolation(nn.Module):
    def __init__(self, scale_factor):
        super(LinearInterpolation, self).__init__()
        self.scale_factor = scale_factor

    def forward(self, x):
        return F.interpolate(x, scale_factor=self.scale_factor, mode='bilinear', align_corners=False)



class SimpleVariationalAutoencoder(nn.Module):
    def __init__(self):
        super(SimpleVariationalAutoencoder, self).__init__()

        self.encoder = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=3,stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.linsys = nn.Sequential(
            nn.Linear(56320, 4096),
            nn.ReLU(),
            nn.Linear(4096, 256))
        
        self.decoder = nn.Sequential(
            LinearInterpolation(scale_factor=2),
            nn.ConvTranspose2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(512, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            LinearInterpolation(scale_factor=2),
            nn.ConvTranspose2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(256, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
             LinearInterpolation(scale_factor=2),
            nn.ConvTranspose2d(128, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            LinearInterpolation(scale_factor=2),
            nn.ConvTranspose2d(64, 1, kernel_size=3, stride=1, padding=1)
        )
        
        self.linsys2 = nn.Sequential(
            nn.Linear(256, 4096),
            nn.ReLU(),
            nn.Linear(4096, 56320))


    def encode(self, x):
        x = self.encoder(x)
        x = torch.flatten(x, start_dim=1)
        mu, sigma = self.linsys(x), self.linsys(x)

        return mu, sigma

    def decode(self, z):
        z = self.linsys2(z)
        z = z.reshape(z.shape[0],512,5,-1)
        x_recon = self.decoder(z)

        return x_recon

    def forward(self, x):
        mu, sigma = self.encode(x)
        z = mu + sigma * torch.randn_like(sigma)

        x_recon = self.decode(z)

        return x_recon, mu, sigma